<span style="font-size: 280%;color:#1155cc"> Pytorch_7_Proste sieci neuronowe robione ręcznie

<span style="font-size: 180%;color:red"> 16.09.2020   


https://github.com/jcjohnson/pytorch-examples#pytorch-custom-nn-modules

W powyższych przykładach musieliśmy ręcznie wdrożyć zarówno przejścia do przodu, jak i do tyłu naszej sieci neuronowej. Ręczne wdrożenie wstecznego przejścia nie jest wielkim problemem dla małej sieci dwuwarstwowej, ale może szybko stać się bardzo kłopotliwe dla dużych złożonych sieci.

In [1]:
import torch

<span style="font-size: 180%;color:#1155cc"> Odpalam karte graficzną GPU

In [2]:
#device = torch.device('cpu') # obliczenia robie na CPU
device = torch.device('cuda') # obliczenia robie na GPU

In [3]:
device

device(type='cuda')

### Dane
- N to wielkość partii; 
- D_in jest wymiarem wejściowym; 
- H jest ukrytym wymiarem; 
- D_out jest wymiarem wyjściowym. 


## <span style="color:red">Przyjmuje 10 zmiennych wynikowych! Coś nowego!</span>

In [4]:
N, D_in, H, D_out = 64, 1000, 100, 10

Powyżej zostały określone parametry aby tensor zmiennych niezależnych i tensor wynikowy były odpowiednie 

x: 64 obserwacji i 1000 zmiennych

y: 64 obserwacji i 10 zmiennych

### Tworzy losowe dane wejściowe i wyjściowe 

In [5]:
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

In [6]:
x.shape

torch.Size([64, 1000])

In [7]:
print('x ',x.size())
print('y ',y.size())

x  torch.Size([64, 1000])
y  torch.Size([64, 10])


### Tworzy wagi losowe dla x i y

In [8]:
w1 = torch.randn(D_in, H, device=device)
w2 = torch.randn(H, D_out, device=device)

In [9]:
w1

tensor([[-1.3308, -1.2433,  0.9052,  ..., -0.8346,  1.0411, -0.2449],
        [-0.7647,  1.5802,  0.5734,  ..., -1.2710, -1.4344, -1.6436],
        [ 2.0647, -1.0230, -0.1434,  ..., -1.1361, -0.4062,  0.0238],
        ...,
        [ 1.6144,  0.7488,  0.5501,  ...,  0.0789,  0.6049, -1.0122],
        [ 0.4601, -0.2690,  0.9715,  ..., -0.9142, -0.4261, -0.3559],
        [-0.3867, -1.4398, -0.1591,  ...,  1.6514, -0.6633, -0.4562]],
       device='cuda:0')

In [10]:
print('w1 dla x: ', w1.shape)
print('w2 dla y: ', w2.shape)

w1 dla x:  torch.Size([1000, 100])
w2 dla y:  torch.Size([100, 10])


### Definiowanie nauki

In [11]:
import time
start_time = time.time() ## pomiar czasu: start pomiaru czasu


learning_rate = 0.0000001     #<= wielkość kroków
for t in range(5000):         #<= ilość epok
 
  h = x.mm(w1)               #<= zwykłe mnożenie macierzy x*w1
  h_relu = h.clamp(min=0)    #<= wyznaczenie ograniczenia do min=0
  y_pred = h_relu.mm(w2)     #<= pomnożenie przez daje predykcję y_pred
#Wyznaczenie straty (loss) jako wskaźnika r2
## Strata Loss jest skalarem i jest przechowywana w tensorze PyTorcha (); 
## możemy uzyskać jego wartość jako liczbę w języku Python za pomocą loss.item ().
  loss = (y_pred - y).pow(2).sum()
  print('Krok:',t, loss.item())        # Dla każdej pentli drukuje wynik r2
  # Backprop do obliczania gradientów w1 i w2 w odniesieniu do strat
  grad_y_pred = 2.0 * (y_pred - y)      #błąd modelu razy 2
  grad_w2 = h_relu.t().mm(grad_y_pred)  #
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h < 0] = 0
  grad_w1 = x.t().mm(grad_h)

  # Zaktualizuj wagi przy użyciu spadku gradientu
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2


print('minutes: ',(time.time() - start_time)/60) ## koniec pomiaru czasu

Krok: 0 32607398.0
Krok: 1 26997256.0
Krok: 2 23349832.0
Krok: 3 20872866.0
Krok: 4 19104468.0
Krok: 5 17772896.0
Krok: 6 16716204.0
Krok: 7 15837425.0
Krok: 8 15077488.0
Krok: 9 14401263.0
Krok: 10 13786748.0
Krok: 11 13220084.0
Krok: 12 12691901.0
Krok: 13 12196152.0
Krok: 14 11728481.0
Krok: 15 11285796.0
Krok: 16 10865800.0
Krok: 17 10466582.0
Krok: 18 10086742.0
Krok: 19 9724987.0
Krok: 20 9380433.0
Krok: 21 9051828.0
Krok: 22 8738243.0
Krok: 23 8438641.0
Krok: 24 8152307.0
Krok: 25 7878454.5
Krok: 26 7616332.0
Krok: 27 7365319.5
Krok: 28 7124909.0
Krok: 29 6894529.0
Krok: 30 6673594.0
Krok: 31 6461741.0
Krok: 32 6258451.5
Krok: 33 6063268.0
Krok: 34 5875838.0
Krok: 35 5695812.5
Krok: 36 5522944.0
Krok: 37 5356902.5
Krok: 38 5197133.0
Krok: 39 5043424.5
Krok: 40 4895469.0
Krok: 41 4752961.0
Krok: 42 4615694.5
Krok: 43 4483457.0
Krok: 44 4356039.0
Krok: 45 4233407.0
Krok: 46 4115171.5
Krok: 47 4001078.75
Krok: 48 3890945.5
Krok: 49 3784627.25
Krok: 50 3681954.5
Krok: 51 3582818.5
K

Krok: 395 19085.896484375
Krok: 396 18891.875
Krok: 397 18700.05859375
Krok: 398 18510.421875
Krok: 399 18322.94921875
Krok: 400 18137.80859375
Krok: 401 17954.869140625
Krok: 402 17774.05078125
Krok: 403 17595.2109375
Krok: 404 17418.3984375
Krok: 405 17243.64453125
Krok: 406 17070.875
Krok: 407 16900.125
Krok: 408 16731.326171875
Krok: 409 16564.328125
Krok: 410 16399.29296875
Krok: 411 16236.0888671875
Krok: 412 16074.720703125
Krok: 413 15915.3125
Krok: 414 15757.572265625
Krok: 415 15601.65234375
Krok: 416 15447.419921875
Krok: 417 15294.935546875
Krok: 418 15144.201171875
Krok: 419 14995.19140625
Krok: 420 14847.787109375
Krok: 421 14701.984375
Krok: 422 14557.8466796875
Krok: 423 14415.2998046875
Krok: 424 14274.376953125
Krok: 425 14134.98046875
Krok: 426 13997.142578125
Krok: 427 13860.7861328125
Krok: 428 13725.9453125
Krok: 429 13592.640625
Krok: 430 13460.798828125
Krok: 431 13330.388671875
Krok: 432 13201.36328125
Krok: 433 13073.771484375
Krok: 434 12947.572265625
Krok: 4

Krok: 850 466.28955078125
Krok: 851 463.08544921875
Krok: 852 459.90374755859375
Krok: 853 456.7474365234375
Krok: 854 453.61328125
Krok: 855 450.50384521484375
Krok: 856 447.4166259765625
Krok: 857 444.34979248046875
Krok: 858 441.3080749511719
Krok: 859 438.2877197265625
Krok: 860 435.2864685058594
Krok: 861 432.3115234375
Krok: 862 429.3568115234375
Krok: 863 426.42498779296875
Krok: 864 423.50946044921875
Krok: 865 420.62298583984375
Krok: 866 417.75091552734375
Krok: 867 414.90374755859375
Krok: 868 412.07440185546875
Krok: 869 409.26556396484375
Krok: 870 406.48114013671875
Krok: 871 403.71484375
Krok: 872 400.966552734375
Krok: 873 398.2410888671875
Krok: 874 395.53564453125
Krok: 875 392.847412109375
Krok: 876 390.1798400878906
Krok: 877 387.5300598144531
Krok: 878 384.90185546875
Krok: 879 382.29052734375
Krok: 880 379.69921875
Krok: 881 377.1279602050781
Krok: 882 374.5735778808594
Krok: 883 372.03936767578125
Krok: 884 369.521240234375
Krok: 885 367.02471923828125
Krok: 886 

Krok: 1301 26.60418701171875
Krok: 1302 26.44619369506836
Krok: 1303 26.28874397277832
Krok: 1304 26.132038116455078
Krok: 1305 25.977386474609375
Krok: 1306 25.82267951965332
Krok: 1307 25.66903305053711
Krok: 1308 25.51703643798828
Krok: 1309 25.36527442932129
Krok: 1310 25.21442222595215
Krok: 1311 25.065433502197266
Krok: 1312 24.91622543334961
Krok: 1313 24.769168853759766
Krok: 1314 24.622211456298828
Krok: 1315 24.476396560668945
Krok: 1316 24.33146858215332
Krok: 1317 24.187381744384766
Krok: 1318 24.044239044189453
Krok: 1319 23.901416778564453
Krok: 1320 23.759937286376953
Krok: 1321 23.619075775146484
Krok: 1322 23.47950553894043
Krok: 1323 23.34027671813965
Krok: 1324 23.20207977294922
Krok: 1325 23.064889907836914
Krok: 1326 22.92853546142578
Krok: 1327 22.792531967163086
Krok: 1328 22.658445358276367
Krok: 1329 22.52400779724121
Krok: 1330 22.39148712158203
Krok: 1331 22.258819580078125
Krok: 1332 22.12726593017578
Krok: 1333 21.996763229370117
Krok: 1334 21.8669376373291

Krok: 1753 1.9776108264923096
Krok: 1754 1.9665653705596924
Krok: 1755 1.9556293487548828
Krok: 1756 1.9449708461761475
Krok: 1757 1.9341676235198975
Krok: 1758 1.923309564590454
Krok: 1759 1.912594199180603
Krok: 1760 1.9019513130187988
Krok: 1761 1.8914010524749756
Krok: 1762 1.8809188604354858
Krok: 1763 1.870542049407959
Krok: 1764 1.8601412773132324
Krok: 1765 1.8498440980911255
Krok: 1766 1.839564561843872
Krok: 1767 1.8293800354003906
Krok: 1768 1.819282054901123
Krok: 1769 1.8091909885406494
Krok: 1770 1.7992088794708252
Krok: 1771 1.7892639636993408
Krok: 1772 1.7793567180633545
Krok: 1773 1.7694556713104248
Krok: 1774 1.759659767150879
Krok: 1775 1.749937891960144
Krok: 1776 1.7402982711791992
Krok: 1777 1.7306324243545532
Krok: 1778 1.7210339307785034
Krok: 1779 1.7115546464920044
Krok: 1780 1.702073574066162
Krok: 1781 1.6926348209381104
Krok: 1782 1.6832504272460938
Krok: 1783 1.6739411354064941
Krok: 1784 1.6646798849105835
Krok: 1785 1.6555473804473877
Krok: 1786 1.64632

...

Krok: 3241 0.007271336857229471
Krok: 3242 0.007256899494677782
Krok: 3243 0.007242349907755852
Krok: 3244 0.00722883827984333
Krok: 3245 0.007214690092951059
Krok: 3246 0.0072031812742352486
Krok: 3247 0.007190239615738392
Krok: 3248 0.007174676284193993
Krok: 3249 0.007164350710809231
Krok: 3250 0.007149752229452133
Krok: 3251 0.007137819193303585
Krok: 3252 0.007122020237147808
Krok: 3253 0.007109291851520538
Krok: 3254 0.007096110377460718
Krok: 3255 0.0070829144679009914
Krok: 3256 0.007071806117892265
Krok: 3257 0.007056036498397589
Krok: 3258 0.007043590769171715
Krok: 3259 0.007028982974588871
Krok: 3260 0.007019025273621082
Krok: 3261 0.007007093634456396
Krok: 3262 0.006992319133132696
Krok: 3263 0.006977754645049572
Krok: 3264 0.006964518688619137
Krok: 3265 0.0069503323175013065
Krok: 3266 0.006941795814782381
Krok: 3267 0.0069265482015907764
Krok: 3268 0.006915688514709473
Krok: 3269 0.00690534757450223
Krok: 3270 0.006892260629683733
Krok: 3271 0.006880261469632387
Krok: 

Krok: 3663 0.0037233217153698206
Krok: 3664 0.0037165614776313305
Krok: 3665 0.003711363300681114
Krok: 3666 0.0037069618701934814
Krok: 3667 0.003702439833432436
Krok: 3668 0.0036969035863876343
Krok: 3669 0.0036909845657646656
Krok: 3670 0.003687599441036582
Krok: 3671 0.003681386820971966
Krok: 3672 0.0036770296283066273
Krok: 3673 0.0036711967550218105
Krok: 3674 0.0036653645802289248
Krok: 3675 0.003660994814708829
Krok: 3676 0.0036572529934346676
Krok: 3677 0.003652665764093399
Krok: 3678 0.0036470447666943073
Krok: 3679 0.0036411508917808533
Krok: 3680 0.0036352945026010275
Krok: 3681 0.003629687475040555
Krok: 3682 0.0036251815035939217
Krok: 3683 0.0036204131320118904
Krok: 3684 0.003616680856794119
Krok: 3685 0.0036119348369538784
Krok: 3686 0.003607107326388359
Krok: 3687 0.003603491699323058
Krok: 3688 0.0036000313702970743
Krok: 3689 0.0035964089911431074
Krok: 3690 0.0035912019666284323
Krok: 3691 0.003585237078368664
Krok: 3692 0.0035812202841043472
Krok: 3693 0.00357750

Krok: 3925 0.0027175331488251686
Krok: 3926 0.0027146642096340656
Krok: 3927 0.0027121941093355417
Krok: 3928 0.0027087016496807337
Krok: 3929 0.0027053076773881912
Krok: 3930 0.002701557707041502
Krok: 3931 0.002698809839785099
Krok: 3932 0.002696005394682288
Krok: 3933 0.0026936193462461233
Krok: 3934 0.0026907450519502163
Krok: 3935 0.002687538508325815
Krok: 3936 0.0026842886582016945
Krok: 3937 0.0026814648881554604
Krok: 3938 0.0026777484454214573
Krok: 3939 0.0026753018610179424
Krok: 3940 0.0026727598160505295
Krok: 3941 0.002670443616807461
Krok: 3942 0.0026678959839046
Krok: 3943 0.0026642431039363146
Krok: 3944 0.0026614749804139137
Krok: 3945 0.002658586483448744
Krok: 3946 0.002653926145285368
Krok: 3947 0.002652334049344063
Krok: 3948 0.002649052068591118
Krok: 3949 0.0026465649716556072
Krok: 3950 0.002644397784024477
Krok: 3951 0.002642315346747637
Krok: 3952 0.002638651058077812
Krok: 3953 0.0026369313709437847
Krok: 3954 0.002635822631418705
Krok: 3955 0.0026328684762

Krok: 4340 0.0018478808924555779
Krok: 4341 0.0018462829757481813
Krok: 4342 0.0018439152045175433
Krok: 4343 0.0018430027412250638
Krok: 4344 0.0018419413827359676
Krok: 4345 0.0018409430049359798
Krok: 4346 0.0018396767554804683
Krok: 4347 0.0018382493872195482
Krok: 4348 0.0018375234212726355
Krok: 4349 0.0018353872001171112
Krok: 4350 0.001833433285355568
Krok: 4351 0.0018317558569833636
Krok: 4352 0.0018301790114492178
Krok: 4353 0.0018288538558408618
Krok: 4354 0.001827271655201912
Krok: 4355 0.0018254003953188658
Krok: 4356 0.001824907143600285
Krok: 4357 0.0018229003762826324
Krok: 4358 0.0018216081662103534
Krok: 4359 0.0018204743973910809
Krok: 4360 0.00181909603998065
Krok: 4361 0.0018181010382249951
Krok: 4362 0.001816463889554143
Krok: 4363 0.001815661322325468
Krok: 4364 0.0018149309325963259
Krok: 4365 0.001813422655686736
Krok: 4366 0.0018109595403075218
Krok: 4367 0.0018096494022756815
Krok: 4368 0.001807957305572927
Krok: 4369 0.0018068325007334352
Krok: 4370 0.001804

Krok: 4601 0.0015267034759745002
Krok: 4602 0.0015260569052770734
Krok: 4603 0.0015241606160998344
Krok: 4604 0.0015227854019030929
Krok: 4605 0.00152190076187253
Krok: 4606 0.0015213373117148876
Krok: 4607 0.0015206665266305208
Krok: 4608 0.001519512850791216
Krok: 4609 0.00151874462608248
Krok: 4610 0.001517806900665164
Krok: 4611 0.0015166846569627523
Krok: 4612 0.001515835989266634
Krok: 4613 0.0015142104821279645
Krok: 4614 0.001513086725026369
Krok: 4615 0.0015126331709325314
Krok: 4616 0.001510953763499856
Krok: 4617 0.0015098376898095012
Krok: 4618 0.0015095293056219816
Krok: 4619 0.0015079571167007089
Krok: 4620 0.0015070312656462193
Krok: 4621 0.0015060256700962782
Krok: 4622 0.0015048558125272393
Krok: 4623 0.0015051639638841152
Krok: 4624 0.0015039498684927821
Krok: 4625 0.0015023746527731419
Krok: 4626 0.0015016017714515328
Krok: 4627 0.0015004578744992614
Krok: 4628 0.001498576020821929
Krok: 4629 0.0014978398103266954
Krok: 4630 0.0014967352617532015
Krok: 4631 0.0014964

CPU minutes:  1.2033316373825074

GPU minutes:  0.7541992386182149